# Training of Model

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load data

In [2]:
trainD = pd.read_csv('Data/train.csv', parse_dates=['date'])

## Slicing for each family and store

In [4]:
#Categorical encoding for dates with categories

#date
Xaux = trainD['date'].astype('category').cat.codes
trainD['date'] = Xaux



fam_unq = trainD['family'].unique()
store_unq = trainD['store_nbr'].unique()
names_col = []
data = []

data.append(trainD['date'].unique())
names_col.append('ds')

for fam in fam_unq:
    trainD_aux = trainD.loc[trainD['family']==fam]
    for store in store_unq:
        #data.append(trainD_aux['id'].loc[trainD['store_nbr']==store].to_list())
        data.append(trainD_aux['sales'].loc[trainD['store_nbr']==store].to_list())
        #names_col.append( fam + "," + str(store) + '_id')
        names_col.append( fam + "," + str(store))

data_t = list(map(list, zip(*data)))
dataDF = pd.DataFrame(data_t,columns=names_col)


## Training model

In [5]:
n_series = len(dataDF)

def shift_n(df,col,n_steps):
    emp_list = df[col].shift(1, fill_value=0).to_list()
    df_ret=pd.DataFrame(emp_list,columns=['0'])
    for i in range(1,n_steps):
        df_ret[str(i)]=df[col].shift(i+1, fill_value=0).to_list()
    return df_ret


#Gradient boosting regressor
from sklearn.ensemble import GradientBoostingRegressor
#MLP Regressor
from sklearn.neural_network import MLPRegressor

n_shifts = 40
regr_list = []
for column in dataDF.iloc[:,1:]:
    data_X = shift_n(dataDF,column,n_shifts)
    data_y = dataDF[column]
    regr_list.append(GradientBoostingRegressor(n_estimators=100, learning_rate=0.2, 
                                random_state=0,loss='squared_error').fit(data_X, data_y))

## Saves the model

In [6]:
import joblib

filename = 'storeSalesPredictionKaggle.sav'
joblib.dump(regr_list, filename)

['storeSalesPredictionKaggle.sav']